## Introduction


### Tutorial Notebook: Vulnscanner

This notebook demonstrates the vulnscanner pipeline step by step:

1. Crawl target site (discover endpoints/forms)
2. Fingerprint technologies/frameworks
3. Harness probes with safe payloads
4. Apply heuristics to interpret signals
5. Reporter generates reproducible evidence

⚠️ Ethics Reminder:
- Written permission required before scanning.
- Scope limited to approved domains/paths.
- Payloads are **non‑destructive** (stored in `data/payloads/`).
- All requests logged in `logs/audit.log`.


##  Setup (Code)

In [ ]:
from src.crawler import Crawler
from src.fingerprinter import Fingerprinter
from src.harness import run_harness
from src.heuristics import HeuristicsEngine
from src.reporter import Reporter

base_url = "https://example.com"  # safe demo target


## Crawler Demo (Code + Markdown)


## Step 1: Crawl

We’ll use BFS (breadth‑first search) to discover forms and endpoints.
Switching to DFS shows traversal order differences.


In [ ]:
crawler = Crawler(base_url, mode="BFS", max_depth=1, max_pages=3)
forms = crawler.crawl()
forms[:2]  # show first few discovered forms


### Fingerprinter Demo (Code + Markdown)


### Step 2: Fingerprinting

We collect technology hints from headers, cookies, script sources, and error signatures.


In [ ]:
fp = Fingerprinter()
for form in forms[:1]:  # fingerprint first endpoint
    fp.fingerprint(form["url"])
fp.report()


### Harness Demo (Code + Markdown)

### Step 3: Harness Probes

We inject harmless payloads (e.g., `' OR '1'='1`, `<script>alert(1)</script>`) into parameters.
We always capture a baseline response for comparison.


In [ ]:
findings = []
for form in forms[:1]:
    url, method = form["url"], form["method"]
    params = {p: "test" for p in form["params"]}
    findings.extend(run_harness(url, method, params))
findings[:2]  # show sample findings


## Step 4: Heuristics

We interpret signals:
- SQLi: error signatures, response diffs, timing anomalies
- XSS: raw reflection, context hints
- CSRF: token absence/staleness, header/cookie checks


In [ ]:
engine = HeuristicsEngine()
for f in findings:
    if "SQLi" in f["type"]:
        engine.detect_sqli(f["url"], f.get("baseline",""), f.get("responses",[]))
    if "XSS" in f["type"]:
        engine.detect_xss(f["url"], f.get("responses",[]))
engine.report()


## Step 5: Reporter

We generate reproducible evidence:
- JSON (machine‑readable)
- HTML (human‑readable with collapsible evidence)
- cURL commands for reproduction
- Risk scoring (exploitability × impact × confidence)


In [ ]:
reporter = Reporter()
for f in findings:
    reporter.add_finding(
        endpoint=f["url"],
        method=f["method"],
        params=f["params"],
        request_excerpt=f.get("request_excerpt",""),
        response_excerpt=f.get("response_excerpt",""),
        diffs=f.get("diffs",{}),
        timing_stats=f.get("timing_stats",{}),
        confidence=f.get("confidence",0.5),
        exploitability=2,
        impact=2
    )

reporter.to_json("reports/findings.json")
reporter.to_html("reports/findings.html")

curl_cmd = reporter.generate_curl("https://example.com/search", "GET", {"q": "' OR '1'='1"})
print(curl_cmd)
